# Nozzle Flow Pressure
This calculates the pressure along the flow of the nozzle from my capstone project using the isentropic flow relations. It outputs a table with the pressure at each point along the Y axis of the nozzle (parallel to the flow). This is then used in an FEA simulation done through ANSYS.

In [2]:
# Importing packages
import pandas as pd
import numpy as np
from scipy.optimize import fsolve

# Initializing
inlet_n = 10
outlet_n = 50
p_total = 600*6894.76

# Setting gammas along nozzle
gamma_inlet = np.linspace(1.225, 1.227, 10)
gamma_throat = 1.227
gamma_outlet = np.linspace(1.227, 1.261, 50)

# Setting y values along nozzle
inlet_n = 10
curve_n = 3
outlet_n = 50
y_inlet = np.linspace(0, 0.01094, inlet_n) #m
y_inlet_curve = np.linspace(0.01094, 0.01324, curve_n)
y_throat = [(0.01324+0.01480)/2]
y_outlet = np.linspace(0.01480, 0.07128, outlet_n) #m
ys = np.concatenate((y_inlet, y_inlet_curve, y_throat, y_outlet))

# Setting radii along nozzle
r_inlet = np.linspace(0.0326, 0.009088, inlet_n)
r_inlet_curve = np.linspace(0.009088, 0.00762, curve_n)
r_throat = 0.00762
r_outlet = np.linspace(0.00762, 0.0176, outlet_n)
rs = np.concatenate((r_inlet, r_inlet_curve, [r_throat], r_outlet))

# Setting areas along nozzle
A_inlet = np.pi*r_inlet**2 #m^2
A_inlet_curve = np.pi*r_inlet_curve**2
A_throat = np.pi*r_throat**2
A_outlet = np.pi*r_outlet**2
As = np.concatenate((A_inlet, A_inlet_curve, [A_throat], A_outlet))

# Defining Area-Mach Relation
def area_mach(x, *data): #a_r is area ratio, g is gamma
    a_r, g = data
    return ((g+1)/2)**(-(g+1)/(2*(g-1))) * (1+(g-1)/2 * x**2)**((g+1)/(2*(g-1)))/x - a_r

# Finding Mach at each point
M_inlet = []
for i in range(inlet_n):
    M_inlet = np.append(M_inlet, fsolve(area_mach, 0.1, args=(A_inlet[i]/A_throat, gamma_inlet[i])))
M_inlet_curve = []
for i in range(curve_n):
    M_inlet_curve = np.append(M_inlet_curve, fsolve(area_mach, 0.1, args=(A_inlet_curve[i]/A_throat, gamma_throat)))
M_throat = [1]
M_outlet = []
for i in range(outlet_n):
    M_outlet = np.append(M_outlet, fsolve(area_mach, 5, args=(A_outlet[i]/A_throat, gamma_outlet[i])))
Ms = np.concatenate((M_inlet, M_inlet_curve, M_throat, M_outlet))

# Finding P at each point
ps_inlet = []
for i in range(inlet_n):
    ps_inlet = np.append(ps_inlet, p_total*(1 + (gamma_inlet[i]-1)/2*M_inlet[i]**2)**(-gamma_inlet[i]/(gamma_inlet[i]-1)))
ps_inlet_curve = []
for i in range(curve_n):
    ps_inlet_curve = np.append(ps_inlet_curve, p_total*(1 + (gamma_throat-1)/2*M_inlet_curve[i]**2)**(-gamma_throat/(gamma_throat-1)))
ps_throat = [p_total*(1 + (gamma_throat-1)/2*M_throat[0]**2)**(-gamma_throat/(gamma_throat-1))]
ps_outlet = []
for i in range(outlet_n):
    ps_outlet = np.append(ps_outlet, p_total*(1 + (gamma_outlet[i]-1)/2*M_outlet[i]**2)**(-gamma_outlet[i]/(gamma_outlet[i]-1)))
ps = np.concatenate((ps_inlet, ps_inlet_curve, ps_throat, ps_outlet))
ps

array([4134218.39711746, 4133170.41351097, 4131545.77969248,
       4128917.45543332, 4124438.01295341, 4116289.04832156,
       4100156.13596267, 4064307.62119857, 3969622.33997423,
       3622629.99125744, 3622629.99125744, 3340812.1908801 ,
       2313643.284776  , 2313643.21629934, 2313643.16830715,
       1697126.6512208 , 1459075.31220974, 1288867.46402746,
       1155179.70319739, 1045378.71855804,  952750.12753226,
        873179.64029116,  803915.01087706,  743002.37865422,
        688994.01944051,  640782.14122938,  597497.57307548,
        558444.58391763,  523057.09585516,  490868.21361134,
        461488.39022086,  434589.39039266,  409892.26181665,
        387158.14789322,  366181.15941291,  346782.76699586,
        328807.3358598 ,  312118.53156735,  296596.39876262,
        282134.96616362,  268640.26751907,  256028.69457282,
        244225.61738094,  233164.22166484,  222784.52366297,
        213032.53113831,  203859.52549161,  195221.44480897,
        187078.35148738,

In [3]:
#Creating DataFrame with all information
df = pd.DataFrame([-ys, ps]).transpose()
df.columns = ["Y Location (m)", "Pressure (Pa)"]
df.to_csv("Optimized Nozzle Information.csv", index=False)